In [13]:
import os
import sys
import json
import re

from glob import glob
from logging import error, warning

from common import CLASS_MAP


def get_docid_class_map():
    docid_class_map = {}
    with open('../data/docid-label-map.tsv') as f:
        for ln, l in enumerate(f, start=1):
            l = l.rstrip()
            fields = l.split('\t')
            docid, class_ = fields
            if docid in docid_class_map:
                raise ValueError('duplicate docid: {}'.format(docid))
            docid_class_map[docid] = class_
    return docid_class_map


def load_biarc_data(dirpath):
    docid_class_map = get_docid_class_map()
    biarcs, classes = [], []
    for fn in glob('{}/*.txt'.format(dirpath)):
        m = re.match(r'^(.*?)-barcs\.', os.path.basename(fn))
        docid = m.group(1)
        class_ = docid_class_map[docid]
        with open(fn) as f:
            data = f.read().strip().split('\t')
        biarcs.append(data)
        classes.append(CLASS_MAP[class_])
    return biarcs, classes


train_biarcs, train_classes = load_biarc_data('../data/split-biarcs/train/')
devel_biarcs, devel_classes = load_biarc_data('../data/split-biarcs/dev/')

In [14]:
from collections import Counter


MIN_EXAMPLES = 25    # filter classes with fewer


class_count = Counter()
for c in train_classes:
    class_count[c] += 1
target_class = set(c for c, v in class_count.items() if v >= MIN_EXAMPLES)


def filter_by_class(data, classes, targets):
    filtered_data, filtered_classes = [], []
    for t, c in zip(data, classes):
        if c in targets:
            filtered_data.append(t)
            filtered_classes.append(c)
    return filtered_data, filtered_classes


train_biarcs, train_classes = filter_by_class(train_biarcs, train_classes, target_class)
devel_biarcs, devel_classes = filter_by_class(devel_biarcs, devel_classes, target_class)

In [15]:
from pprint import pprint

def class_counts(classes):
    counter = Counter()
    for c in classes:
        counter[c] += 1
    return counter


pprint(class_counts(train_classes))

Counter({'MT/Gen': 296,
         'D-Sell': 243,
         'News': 184,
         'B-Personal': 139,
         'D-Thing': 103,
         'How-to': 86,
         'Religious': 75,
         'B-Personal-Opinion': 70,
         'Forums': 65,
         'B-Community': 49,
         'Reviews': 44,
         'A-Encyclopedia': 35,
         'Sports': 30,
         'Editorials': 26})


In [17]:
def biarcs_to_text(biarcs):
    return ['\t'.join(b) for b in biarcs]


train_texts = biarcs_to_text(train_biarcs)
devel_texts = biarcs_to_text(devel_biarcs)

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC


def biarc_tokenizer(text):
    return text.split('\t')


vectorizer = TfidfVectorizer(analyzer='word', tokenizer=biarc_tokenizer, lowercase=False, ngram_range=(1,1))
vectorizer.fit(train_texts)

train_X = vectorizer.transform(train_texts)
devel_X = vectorizer.transform(devel_texts)

classifier = LinearSVC(C=1.0)
classifier.fit(train_X, train_classes)

classifier.score(devel_X, devel_classes)

0.5436893203883495

In [30]:
import eli5


eli5.show_weights(classifier, vec=vectorizer, top=100)

In [29]:
import numpy as np

from sklearn.metrics import confusion_matrix
from pandas import DataFrame


pred_Y = classifier.predict(devel_X)
cm = confusion_matrix(devel_classes, pred_Y)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]    # normalize
df = DataFrame(cm * 100, index=classifier.classes_, columns=classifier.classes_)
df.round(2)

,A-Encyclopedia,B-Community,B-Personal,B-Personal-Opinion,D-Sell,D-Thing,Editorials,Forums,How-to,MT/Gen,News,Religious,Reviews,Sports
A-Encyclopedia,0.0,0.00,0.00,0.00,20.00,0.00,0.0,0.00,0.00,0.00,80.00,0.00,0.00,0.00
B-Community,0.0,0.00,25.00,0.00,37.50,12.50,0.0,0.00,0.00,0.00,25.00,0.00,0.00,0.00
B-Personal,5.0,0.00,70.00,0.00,5.00,5.00,0.0,5.00,0.00,5.00,0.00,5.00,0.00,0.00
B-Personal-Opinion,0.0,0.00,30.00,30.00,0.00,0.00,0.0,10.00,0.00,0.00,10.00,10.00,10.00,0.00
D-Sell,0.0,0.00,9.09,0.00,54.55,6.06,0.0,0.00,3.03,12.12,12.12,0.00,3.03,0.00
D-Thing,0.0,0.00,0.00,0.00,46.67,13.33,0.0,0.00,0.00,13.33,26.67,0.00,0.00,0.00
Editorials,0.0,0.00,25.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,50.00,25.00,0.00,0.00
Forums,0.0,0.00,44.44,11.11,11.11,0.00,0.0,22.22,0.00,11.11,0.00,0.00,0.00,0.00
How-to,0.0,0.00,16.67,0.00,16.67,8.33,0.0,0.00,33.33,8.33,16.67,0.00,0.00,0.00
MT/Gen,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,100.00,0.00,0.00,0.00,0.00
